### **Naive Bayes Classifier**

**[Probability Overview]** - 머신러닝의 학습방법들
- Gradient descent based learning
- Probability theory based learning - 이번에 배울거
- Information theory based learning
- Distance similarity based learning

**Probability**
- 이산형 값
> ##### $P(X) = \frac{count(Event_X)}{count(ALL_{Event})}$

- 연속형 값
> $P(-\infty < x < \infty) = \int_{-\infty}^{\infty} f(x)dx = 1$
<br><br><br>

**Basic concepts of probability**
> $0 \le P(E) \le 1$ <br>
> $P(S) = \sum_{i=1}^N P(E_i) = 1$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;if all $E_i$ are independent
<br><br><br>

**Conditional probability**
> $P(A|B) = \frac{P(A \cap B)}{P(B)}$

<br><br>
**[Bayes's Theorem]**
- 경험에 의한 확률 업데이트
- 주어진 사전 확률에서 특정 사건 발생을 통해 사후 확률로 계속해서 업데이트(ex.개표방송)
- 빈도주의 vs 베이즈주의
- 객관적 확률은 존재하지 않는다!

**Probability updated**
- 특정 사건이 일어남에 따라 확률을 업데이트(ex. 퀸찾기 카드게임)

**Bayes's theorem**
> $P(A|B) = \frac{P(A \cap B)}{P(B)}$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$P(B|A) = \frac{P(A \cap B)}{P(A)}$ <br>
> $P(A \cap B) = P(B)P(A|B) = P(A)|P(B|A)$ <br><br>
> $P(A|B) = \frac{P(A \cap B)}{P(B)} = \frac{P(A)P(B|A)}{P(B)}$
- 다시말해서, 사건B가 일어났을 때의 사건A가 일어날 확률은, 사건B가 일어났을때, 사건A가 일어날 확률은 주어져있고, 사건A가 일어났을때 사건B가 일어날 확률을 거기에 곱해주면 된다

> $P(H|D) = \frac{P(H)P(D|H)}{P(D)}$ <br>
(H is Class &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; D is Data)
- P(H|D) : 사후확률, 어떤 데이터가 주어졌을 때의 가설이 발생할(발생하지않을) 확률 (H : 가설함수)
- P(D) : 전체확률, 데이터가 발생할 확률(Evidence)
- P(H) : 사전확률, 예) 1000개중 $y_1$= 700, $y_2$=300개면 P($H_1$)=700/1000
- 결국은 주어진 데이터(D) X를 통해 클래스(D) Y를 찾는 작업
- 나중에는 P(D)는 고정값이 되고, P(H)는 주어져서 결국 P(D|H), 우도를 찾는게 가장 중요해진다.

<br><br>
**[Naive Bayes Classifier]**<br><br>
예제로 풀어보기 - Viagra 스팸 필터기
- Viagra라는 단어의 유무를 통해 스팸 여부 확인
- Viagra 단어가 들어가면 무조건 스팸?
- 어느정도 확률로 스팸이라고 해야할까? <br><br>
![img](https://ifh.cc/g/3vSPxP.png)

- $P(spam | viagra) = \frac{P(spam)P(viagra|spam)}{P(viagra)}$

In [95]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [96]:
viagra_spam = {'viagra': [1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1],
               'spam': [
                   1,0,0,0,0,0,1,0,1,0, 0,0,0,0,0,0,0,1,1,1
               ]}

df = pd.DataFrame(viagra_spam, columns=['viagra', 'spam'])
df.head()

,viagra,spam
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0


In [97]:
np_data = df.values
np_data

array([[1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 1],
       [1, 1]], dtype=int64)

- P(viagra), P(spam), P(V $\cap$ S), P($N^c \cap S$) 구하기

In [98]:
p_viagra = sum(np_data[:, 0] == 1) / len(np_data)
p_spam = sum(np_data[:, 1] == 1) / len(np_data)
p_v_cap_s = sum((np_data[:, 0] == 1) & (np_data[:, 1] == 1)) / len(np_data)
p_n_v_cap_s = sum((np_data[:, 0] == 0) & (np_data[:, 1]== 1)) / len(np_data)

- P(Spam | Viagra)

In [99]:
p_spam * (p_v_cap_s / p_spam) / p_viagra

0.5

- P(Spam | ~Viagra)

In [100]:
p_spam * (p_n_v_cap_s / p_spam) / (1-p_viagra)

0.2142857142857143

<br><br>
**제대로된 스팸 필터기를 만들어보자**
- Viagra 단어외에 영향을 주는 단어들은?
- 오히려 스팸을 제외해주는 단어는 어떻게 찾을까?
- 한번에 여러 단어들을 고려하는 필터기를 만들어보자

**Feature의 확장**<br><br>
$P(spam | viagra)$ &nbsp;&nbsp;( spam : y &nbsp;&nbsp;&nbsp;&nbsp; viagra : x )<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\Downarrow$<br>
$P(spam | viagra, hello, lucky, marketing, \cdots)$
- y는 그대로 1또는 0을 가지지만, x는 $x_1, x_2, x_3 \cdots $로 확장됨
- 변수가 많을 때, 조건부 확률의 변화

**[ Multivariate multiplication rule ]**
> $P(Y | X_1 \cap X_2) = \frac{P(Y \cap X_1 \cap X_2)}{P(X_1 \cap X_2)}$<br>
> $P(Y \cap X_1 \cap X_2) = P(Y | X_1, X_2) P(X_1 \cap X_2)$ <br><br>
> $P(X_1, X_2, X_3, \cdots, X_n)$<br>
> $= P(X_1)P(X_2|X_1)P(X_3|X_1, X_2) \cdots P(X_n|X_1 \cdots X_{x-1})$ 

**Problems!**
- 변수가 늘어날수록 급격히 계산이 어려워짐
- Feature의 차원이 증가하면 Sparse Vector가 생성 $\rightarrow$ 확률이 0이 되는 값이 늘어남

#### **[ Naive Bayes Classifier ]**
- 접근방향 : 복잡하게 하지만고 단순(naive)하게 해결하자
- 각 변수의 관계가 독립임을 가정
- 계산이 용이해지고, 성능이 생각보다 좋음..!

**Joint Probability**
> $P(A \cap B) = P(A)P(B)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if A and B are independent<br><br>
> $P(Y|X_1 \cap X_2) = \frac{P(Y)P(X_1 \cap X_2 | Y)}{P(X_1 \cap X_2)}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$= \frac{P(Y)P(X_1|Y)P(X_2|Y)}{P(X_1)P(X_2)}$
- 이전의 Multivariate multiplication rule보다는 더 간결해짐

**Naive Bayes Classifier**
> ##### $P(Y|X_1 \cap X_2) = \frac{P(Y) P(X_1 \cap X_2 | Y)}{P(X_1 \cap X_2)} = \frac{P(Y)P(X_1|Y)P(X_2|Y)}{P(X_1)P(X_2)}$<br><br>
> ##### <span style='background-color:#fff5b1'>$P(Y_c | X_1, \cdots,X_n) > = \frac{P(Y_c) \prod_{i=1}^n P(X_i |Y_c)}{\prod_{i=1}^n P(X_i)}$</span> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;$Y_c$ : label ($Y_c$ = 0 or 1)
- $X_1$부터 $X_n$까지의 Data가 주어졌을 때, P(Y=1)과 P(Y=0)중에 더 큰 값을 채택하여 모든 데이터마다 진행

**Issue!**
- 너무 많은 확률값$(0 \le P \le 1) \rightarrow$ 0에 수렴하게 되는문제
- 곱하지 말고 더하자 $\rightarrow$ log
<br><br>
> $log({P(Y_c)\prod_{i=1}^n P(X_i|Y_c)}) = log P(Y_c) + \sum_{i=1}^n log P(X_i |Y_c)$
> ##### <span style='background-color:#fff5b1'>$P(Y_c | X_1, \cdots, X_n) = \frac{P(Y_c) \prod_{i=1}^n P(X_i|Y_c)}{\prod_{i=1}^n P(X_i)}$</span> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $Y_c$ : label ($Y_c$ = 0 or 1)
- 분모에서 $X_i$는 피쳐이기 때문에 prod의 결과값은 고정되어있어서 중요하지 않다.<br>
$\divideontimes$ Likelihood : 가능도 <br>
<br><br>
- 확률이 0인 변수들이 존재함 $\rightarrow$ 전체값 0
- 작게나마 확률이 나올 수 있도록변경 $\rightarrow$ **스무딩**
> #### $P(X|Y) = \frac{count(X \cap Y) + k}{count(Y) + (k\;*\;|number\;of\;class|)}$
- 분모분자에 k라는 상수값을 임의로 넣어서 값을 좀 더 평탄하게(generalize) 해준다. 다만 count값들에 비해 k가 커버리면 차이가 더 흐려질수도 있으니 주의
- 스무딩 기법을 사용하면 확률이 0으로 산출되지도 않고 generalize의 효과도 볼수있다.
- binary한 분류문제에서는 Y=1 or 0이 될테니까 number of class=2가 된다
> ##### <span style='background-color:#fff5b1'> $log{P(Y_c)\prod_{i=1}^n P(X_i|Y_c)} = log P(Y_c) + \sum_{i=1}^n logP(X_i | Y_c)$</span>
- 최종식. 우변에서 sum항에 스무딩을 먹여줘서 0인 값이 없도록 한다.
<br><br>
- NB의 핵심 : 최종 식 도출 + 스무딩

<br><br>
### **NB Classifier Implementation**

**Dataset - German Credit**
- 대출 사기인가 아닌가를 예측하는 문제
- 데이터를 NB에 맞도록 간단하게 변환
- Binary 데이터들로 이루어진 대출 사기 데이터

**Data Load, Preprocessing**

In [101]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [102]:
df = pd.read_csv('https://raw.githubusercontent.com/TeamLab/machine_learning_from_scratch_with_python/master/code/ch11/fraud.csv')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [103]:
del df["ID"]

Y_data = df.pop("Fraud")
Y_data = Y_data.as_matrix()
Y_data

array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [104]:
df.head()

,History,CoApplicant,Accommodation
0,current,none,own
1,paid,none,own
2,paid,none,own
3,paid,guarantor,rent
4,arrears,none,own


**Preprocessing**
- History_arrears = x1, History_current = x2 ... 이런식으로 onehot encoding

In [105]:
x_df = pd.get_dummies(df)
x_df.head()

,History_arrears,History_current,History_none,History_paid,CoApplicant_coapplicant,CoApplicant_guarantor,CoApplicant_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,1,0
3,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,1,0


- onehot encoding의 결과로 x1부터 x10까지의 feature들이 나왔음
- 따라서 x의 벡터들이 주어졌을 때, P(y=1)과 P(y=0)중 더 큰걸 선택할수 있도록 classifier를 만들어줄거임

In [106]:
x_data = x_df.as_matrix()
x_data

array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

**Modeling**
##### $P(Y_c | X_1, \cdots, X_n) = \frac{P(Y_c) \prod_{i=1}^n P(X_i|Y_c)}{\prod_{i=1}^n P(X_i)}$<br>
- 위 식을 따르는 Naive Bayes's Classifier를 만든다.
- $x_1, \cdots ,x_n$ 은 $x_1, \cdots ,x_{10}$ 까지
- 분모는 모든 y의 값에 적용되니 무시해도된다.

In [107]:
P_Y_True = sum(Y_data == True) / len(Y_data)
P_Y_False = 1 - P_Y_True

P_Y_True, P_Y_False

(0.3, 0.7)

- Y의 Index : 사기가 있는것과 없던것의 인덱스 구분
- np.where() : 괄호안의 조건에 성립하는것의 인덱스값을 반환

In [108]:
# 사기 당한 인덱스들
ix_Y_True = np.where(Y_data)
# 사기 안당한 인덱스들
ix_Y_False = np.where(Y_data == False)

ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12], dtype=int64),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19],
        dtype=int64),))

##### $P(X_i | Y_c) = \frac{Count(X_i \cap Y = 1)}{Count(Y=1)} \cdots$ (Y=0일때도)
- (x_data[ix_Y_True].sum(axis=0)) 부분 설명<br><br>
![img](https://ifh.cc/g/52Dvnx.png)
- 위에서 뽑은 인덱스들의 행만 뽑은 뒤 각 피쳐별로 위에서 아래로(axis=0) 쭉 sum 진행.
- 그 결과값에 인덱스의 갯수로 나눠줘서 위의 식 구현


In [109]:
p_x_y_true = (x_data[ix_Y_True].sum(axis=0)) / sum(Y_data == True) # 분모 : count(Y=1)을 의미
p_x_y_false = (x_data[ix_Y_False].sum(axis=0)) / sum(Y_data == False) # 분모 : count(Y=0)을 의미

p_x_y_true, p_x_y_false

(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
        0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))

**Classifier**
- 사실 아래 식에서 우변의 각 항에 log를 취해주거나 그게 아니면 저 두 항을 서로 곱해줘야하는데 값의 큰 차이가 없어서 더해주면서 진행

In [110]:
x_test = [0, 1, 0, 0, 0, 1, 0, 0, 1, 0]

p_y_true_test = P_Y_True + p_x_y_true.dot(x_test) # P(Y = 1)
p_y_false_test = P_Y_False + p_x_y_false.dot(x_test) # P(Y = 0)

p_y_true_test, p_y_false_test
# P(Y = 1) < P(Y = 0)

(1.6333333333333333, 1.7714285714285714)

In [111]:
p_y_true_test < p_y_false_test
# 따라서 x_test 데이터에 대해서는 대출사기가 아니라고 하는것이 맞다

True

### **Multinomial Naive Bayes**
- X값이 Binary가 아니라 1이상의 값을 가지는 문제
- 일반적으로 Text 문제를 분류할 때 많이 쓰임
- 단어의 존재 유무가 아닌 단어의 **출현횟수** Feature로
- 구성된 벡터가 sparse할 때 많이 쓰임 ([0 0 0 1 0 0 7 0 0 1 0 0]같은)

**[ Text를 문자로 표현하는 방법 ]**<br><br>
**문자를 Vector로 - One hot Encoding**
- 하나의 단어를 Vector의 Index로 인식, 단어 존재시 1 없으면 0<br><br>
![img](https://ifh.cc/g/bSmZjf.png)
- 어떤 단어 자체를 인덱스를 먹여서 존재여부로 표현
<br><br>
- 하지만 한 문장에서 한번에 여러 단어가 나올때는..?

**[ Bag of words ]**
- 단어별(One-hot)로 인덱스를 부여해서 한 문장(또는 문서)의 단어의 개수를 Vector로 표현
- 순서의 정보는 표현할 수 없지만 성능이 강력해서 주로 이용됨<br><br>
![img](https://ifh.cc/g/9Dwk92.png)<br>
(문장에서 the가 두번나오니 the의 초록색 박스의 숫자는 2가 되어야함 )<br>

- Bag of words 예시<br>
![img](https://ifh.cc/g/gqFPVY.png)

**[ Multinomial Naive Bayes ]**
- 식 계산하는 방식이 다름<br>

> $\prod_{i=1}^n P(X_i | Y_c)$에서의 $P(X_i | Y_c)$<br>
> ### $ \Rightarrow P(X_i | Y_c) = \frac{\sum t f(x_i, d \in Y_c) + \alpha}{\sum N_{d \in Y_c} + \alpha \centerdot V}$

- $\sum t f(x_i, d \in Y_c)$ : 특정 클래스에서 특정 x의 갯수들의 합
- $\sum N_{d \in y_c}$ : 클래스에 속한 모든 단어의 갯수의 합
- $x_i$ : $Y_c$가 특정상황일때, feature의 출현횟수
- V : feature의 갯수(단어의 갯수)
- $\alpha$ : 스무딩 파라메터<br><br><br>

- 예제 풀어보기<br><br>
![img](https://ifh.cc/g/zJMXB9.png)<br><br>
> ##### $P(X_i | Y_c) = \frac{\sum tf(x_i, d \in Y_c) + \alpha}{\sum N_{d \in Y_c} + \alpha \centerdot V} \cdots\cdots \mathit{1}$
> ##### $P(Y_c | X_1, \cdots, X_n) = \frac{P(Y_c) \prod_{i=1}^n P(X_i|Y_c)}{\prod_{i=1}^n P(X_i)} \cdots\cdots \mathit{2}$<br>
**[ Train ]**<br>
- P(c) = 3/4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;P(j) = 1/4<br>
$\alpha=2$로 가정

- $X_1$ = Chinese<br>$X_2$ = Beijing<br>$X_3$ = Shanghai<br>$X_4$ = Macao<br>$X_5$ = Tokyo<br>$X_6$ = Japan

식 1의 $P(X_i|Y_c)$구하기 위해서 $P(X_i | c)$과$P(X_i | j)$를 구할것임<br><br>
**1**. $P(X_1 | c)$ : class가 china일 때, $X_1$이 들어있는 확률 
- V = 6
- $N_{d \in c}$ = 8
- $\sum tf(x_1, d \in c)$ = china클래스 안에서 Chinese의 갯수 = 5
- 따라서, $P(X_1 | c) = \frac{5 + 2}{8 + 12}$

**2**. $P(X_i | j)$ : class가 japan일 때, $X_1$이 들어있는 확률
- V = 6
- $N_{d \in j}$ = 3
- $\sum tf(x_1, d \in c)$ = japan클래스 안에서 Chinese의 갯수 = 1
- 따라서, $P(X_1 | j) = \frac{1 + 2}{3 + 12}$
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\vdots$<br>
- $X_6$까지 모두 구한다음 $\prod_{i=1}^n P(X_i| c)$와 $\prod_{i=1}^n P(X_i| j)$를 해서 다 곱해준 뒤 식 2를 구할 때 활용한다.<br><br><br>

**[ Test ]**
- 식 2를 활용하여 주어진 데이터가 class c에 속할지 j에 속할지 구분해야된다.
- Test데이터에 Chinese, Tokyo, Japan이라는 단어가 있으니 $X_1,\ X_5,\ X_6$이 있다고 본다.
- 따라서 최종적으로 식2를 활용하여,
> #### $P(c|X_1, \cdots, X_n) = \frac{P(c) * P(X_1|c)* P(X_5|c)* P(X_6|c)}{\prod_{i=1}^n P(X_i)}$
> #### $P(j|X_1, \cdots, X_n) = \frac{P(j) * P(X_1|j)* P(X_5|j)* P(X_6|j)}{\prod_{i=1}^n P(X_i)}$
- 위 두 식의 결과 중 더 큰값이 나오는 쪽으로 분류하면 된다.



<br><br><br>
#### **[ Gaussian NB ]**
- Category데이터가 아닌 Continuous한 경우에 적용되는 NB
- Continuous 데이터의 적용을 위해 y의 분포를 **정규분포(gaussian)으로 가정**함
- 확률밀도 함수 상의 해당 값 x가 나올 확률로 NB를 구현함

> #### $P(x_i | Y_c) = \frac{1}{\sqrt{2\pi \sigma^2_{Y_c}}}exp(- \frac{(x_i - \mu Y_c)^2}{2\sigma^2_{Y_c}})$
- 특정 $Y_c$에 대한 $X_i$의 평균 표준편차를 $\mu$, $\sigma$에 대입
- 예) Y=1일때, $X_1$이라는 피쳐의 $\mu$값과 $\sigma$값을 위 식에 대응시켜줌


<br><br>
**예제 풀어보기**
<br><br>
- 키, 몸무게, 발 사이즈($X_1, X_2, X_3$)를 보고 남자인지 여자인지 예측하는 문제<br>
![img](https://ifh.cc/g/xHX3v9.png)<br><br>
- 남자or 여자일 경우(Y=1 or 0)의 각 피쳐들($X_i$)의 평균, 분산값들
- 아래 값들을 위의 식에 그냥 대입해주면 된다.
![img](https://ifh.cc/g/tsQKla.png)
<br><br><br>
- 그 후에 test데이터가 등장하면 예측해준다.<br>
![img](https://ifh.cc/g/w1YYCF.png)

- 위의 식 참고해서 확률이 더 큰 값 구해준다.
- 여기선 $\alpha$는 없다.

#### **Naive Bayes with Sklearn**

**[ CountVectorizer in Scikit-learn ]**
- 문서에서 Bag of Words Vector를 뽑아주는 class
- 단어를 어떻게 뽑을지 정해주는 기능
- 다른 전처리 모듈처럼 생성 $\rightarrow$ 적용의 과정을 거침
- A, the 같은 조사들은 **stop word**로써 CountVectorizer내에서 알아서 제거해줌

In [112]:
from sklearn.feature_extraction.text import CountVectorizer

- 출력값의 앞에서부터 인덱스를 부여받았다.($X_1,X_2,X_3, \cdots$)

In [113]:
y_example_text = ["Sports", "Not sports","Sports","Sports","Not sports"]
y_example = [1 if c=="Sports" else 0 for c in y_example_text ]
text_example = ["A great game game", "The The election was over",
                "Very clean game match",
                "A clean but forgettable game game","It was a close election", ]

countvect_example = CountVectorizer()
X_example = countvect_example.fit_transform(text_example)
countvect_example.get_feature_names()[:8]

['but', 'clean', 'close', 'election', 'forgettable', 'game', 'great', 'it']

- 출력값의 첫번째 줄은 A great game game에서의 해당 인덱스가 몇개의 단어를 가지고 있는지 나타냄

In [114]:
X_example.toarray()

array([[0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]], dtype=int64)

**[ NB classifier family in scikit-learn ]**
- Scikit-learn에서 제공하는 NB classifier
- Bernoulli Naive Bayes
- Multinomial Naive Bayes
- Gausiian Naive Bayes

**[ Bernoulli Naive Bayes ]**<br><br>
**Parameters**<br>
- alpha : 스무딩 파라미터
- binarize : threshold, 단어등장 갯수를 세는 경우에 몇개 이상(이하)을 1(0)이라 표현할것인지
- fit_prior : 학습으로 class를 나눌것이냐(중요x)
- class_prior : 미리 class를 지정해서 나눌것이냐(중요x)<br>

**Attributes**<br>
- class_log_prior : $log$처리된 $P(Y_c)$출력
- feature_log_prob : $log$처리된 Likelihood, $logP(X_i|Y)$
- class_count_ : class 몇개인지
- feature_count_ : feature 몇개인지

In [115]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB(binarize=0) # 1개만 출연해도 1로
clf.fit(X_example, y_example)
# Y=1일때 값, Y=0일 때 값
clf.class_log_prior_

array([-0.91629073, -0.51082562])

In [116]:
clf.class_count_

array([2., 3.])

In [117]:
clf.feature_log_prob_
# [Y=0일때 , 각 피쳐들의 logP가 얼마인지], [Y=1일 때 ...]

array([[-1.38629436, -1.38629436, -0.69314718, -0.28768207, -1.38629436,
        -1.38629436, -1.38629436, -0.69314718, -1.38629436, -0.69314718,
        -0.69314718, -1.38629436, -0.28768207],
       [-0.91629073, -0.51082562, -1.60943791, -1.60943791, -0.91629073,
        -0.22314355, -0.91629073, -1.60943791, -0.91629073, -1.60943791,
        -1.60943791, -0.91629073, -1.60943791]])

In [118]:
clf.feature_count_
# [Y=0일때, 각 피쳐들이 몇번씩 등장했는지], [Y=1일 때]

array([[0., 0., 1., 2., 0., 0., 0., 1., 0., 1., 1., 0., 2.],
       [1., 2., 0., 0., 1., 3., 1., 0., 1., 0., 0., 1., 0.]])

**[ Multinomial Naive Bayes ]**<br><br>
**Parameters**<br>
- alpha : 스무딩 파라미터
- fit_prior : 학습으로 class를 나눌것이냐(중요x)
- class_prior : 미리 class를 지정해서 나눌것이냐(중요x)<br>


**Attributes**<br>
- class_log_prior_ : $log$처리된 $P(Y_c)$출력
- intercept_ : LR에서의 절편, class_log_prior와 대동소이
- feature_log_prob : $log$처리된 Likelihood, $logP(X_i|Y)$
- coef_ : feature_log_prob와 대동소이
- class_count_ : class 몇개인지
- feature_count_ : feature 몇개인지

In [119]:
from sklearn.naive_bayes import MultinomialNB

In [120]:
clf = MultinomialNB()
clf.fit(X_example, y_example)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [121]:
print(clf.class_log_prior_)
print(clf.feature_log_prob_)
print(clf.class_count_)
print(clf.feature_count_)

print(clf.coef_)
print(clf.intercept_)

[-0.91629073 -0.51082562]
[[-3.09104245 -3.09104245 -2.39789527 -1.99243016 -3.09104245 -3.09104245
  -3.09104245 -2.39789527 -3.09104245 -2.39789527 -1.99243016 -3.09104245
  -1.99243016]
 [-2.52572864 -2.12026354 -3.21887582 -3.21887582 -2.52572864 -1.42711636
  -2.52572864 -3.21887582 -2.52572864 -3.21887582 -3.21887582 -2.52572864
  -3.21887582]]
[2. 3.]
[[0. 0. 1. 2. 0. 0. 0. 1. 0. 1. 2. 0. 2.]
 [1. 2. 0. 0. 1. 5. 1. 0. 1. 0. 0. 1. 0.]]
[[-2.52572864 -2.12026354 -3.21887582 -3.21887582 -2.52572864 -1.42711636
  -2.52572864 -3.21887582 -2.52572864 -3.21887582 -3.21887582 -2.52572864
  -3.21887582]]
[-0.51082562]


**[ Gaussian Naive Bayes ]**<br><br>
**Parameters**<br>
- priors : 학습해서 분류할거냐 안그럴거냐


**Attributes**<br>
- class_prior_ : $P(Y_c)$출력
- class_count_ : class 몇개인지
- theta_ : $\mu$
- sigma_ : $\sigma$

In [122]:
from sklearn.naive_bayes import GaussianNB

In [123]:
X_example

<5x13 sparse matrix of type '<class 'numpy.int64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [124]:
clf = GaussianNB()
# GaussianNB같은 경우는 sparse한 데이터 형태를 싫어해서 numpy로 바꿔주는 toarray를 적용시켜준다.
clf.fit(X_example.toarray(), y_example)

GaussianNB(priors=None)

In [125]:
print(clf.class_count_)
print(clf.class_prior_)
print(clf.theta_)
print(clf.sigma_)

[2. 3.]
[0.4 0.6]
[[0.         0.         0.5        1.         0.         0.
  0.         0.5        0.         0.5        1.         0.
  1.        ]
 [0.33333333 0.66666667 0.         0.         0.33333333 1.66666667
  0.33333333 0.         0.33333333 0.         0.         0.33333333
  0.        ]]
[[8.00000000e-10 8.00000000e-10 2.50000001e-01 8.00000000e-10
  8.00000000e-10 8.00000000e-10 8.00000000e-10 2.50000001e-01
  8.00000000e-10 2.50000001e-01 1.00000000e+00 8.00000000e-10
  8.00000000e-10]
 [2.22222223e-01 2.22222223e-01 8.00000000e-10 8.00000000e-10
  2.22222223e-01 2.22222223e-01 2.22222223e-01 8.00000000e-10
  2.22222223e-01 8.00000000e-10 8.00000000e-10 2.22222223e-01
  8.00000000e-10]]


<br><br>
#### **News group classification**
- 앞서 배운 내용들을 text데이터를 분류하는 문제에 적용해보자

**[ 20 newsgroups Dataset ]**
- 대표적인 Text분류 Toy dataset
- 20개의 뉴스 텍스트 데이터를 분류하라 !
- Multiclass classification의 대표적 문제
- 약 20,000개의 news document 존재<br><br>

**[ Process for text classification ]**<br><br>
![img](https://ifh.cc/g/Zczvxy.png)
<br><br>

**[ 텍스트 전처리 ]**<br><br>
**Data cleansing**
- 어떤 Text를 남길것인가?
- 특수 문자는 남겨둘 것인가? ('Good' vs 'Good?')
- 숫자는 어떻게 할것인가? ('4 seasons' vs 'seasons')
- 특수문자를 제거한다면 공백은? (I'm $\rightarrow$ Im or I m)
- 이메일, ip주소 등 특수 문자들의 처리는?<br><br><br>
- **텍스트 전처리 과정**<br><br>
![img](https://ifh.cc/g/LvvASt.png)
- **Tokenizer** : 단어 자르기
- **Stop Word Filtering** : 해당 문장을 특성할 수 있도록 너무 흔한 단어는 없앰
- **Steaming** : 어간(활용어가 활용할 때에 변하지 않는 부분)만 남김<br><br>

**[ Vectorize ]**<br><br>
**ngrams** : 한 번에 몇개의 단어를 묶을것인가?<br><br>
![img](https://ifh.cc/g/cPLS3G.png)
- N = 1 : tokenize한 것 처럼 한 단어씩 분리
- N = 2 : 두단어씩 묶어서 분리
- 분리된 단어는 하나의 feature가 된다.<br><br><br>

**TF-IDF**(Term Frequency - Inverse Document Frequency)<br>
 : 전체 문서에서 많이 나오는 단어는 중요도를 낮추고, 특정 문서에서만 많이 나오면 중요도를 올림
> $w_{i, j} = tf_{i,j} \times log(\frac{N}{df_i})$
> - i : 단어의 index
> - j : 문서의 index
> - $tf_{i,j} : Term frequency : j문서에 i단어가 존재한 횟수$
> - N : 전체 문서
> - $df_i$ : i단어가 있는 문서의 갯수

- 위의 식대로 한 단어마다 TF-IDF를 먹임<br><br>
<예시><br>
![img](https://ifh.cc/g/Tv6qNz.png)<br>
- 단어가 4개, 문서가 3개 있는 경우
- car를 예로들면, $27 \times log(\frac{3}{3})$이기 때문에
- x = 1 일때 y = 0이 도출되어서 전체 문서의 입장에서 car는 중요한 단어가 아니라고 판단<br>
- 나머지는 아래 그래프처럼 도출되어 y값이 클수록 중요도가 높다고 판단
![img](https://ifh.cc/g/SGXDGs.png)<br><br><br><br>


In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import re
%matplotlib inline

**Data Loading**
- Scikit-learn 내부 모듈을 활용함

In [127]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

In [128]:
# 하키에 관한 내용
print(news.data[0])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [129]:
# news.target_names에서 10번째에 오는 데이터로 진행
news.target[0]

10

In [130]:
# 하키에 관한 내용
news.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

**Dataset**
- 18846개의 데이터

In [134]:
news_df = pd.DataFrame({'News' : news.data, 'Target' : news.target})

In [135]:
news_df.head()

,News,Target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4


**Data tagging**

In [136]:
# Target 데이터 -> 문자 라벨링(뉴스마다 어떤 뉴스인지 보기 편하도록 만들기 위해서)
def word_labeling(lst, df):
    for idx, name in enumerate(lst):
        target_data = df['Target']
        for idx_, num_label in enumerate(target_data):
            if num_label == idx:
                df.loc[idx_, 'Target'] = name
    return df
news_df = word_labeling(news['target_names'], news_df)
news_df.head()

,News,Target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,rec.sport.hockey
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,comp.sys.ibm.pc.hardware
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,talk.politics.mideast
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,comp.sys.ibm.pc.hardware
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,comp.sys.mac.hardware


**Data cleansing**
- 이메일 제거
- 불필요 숫자 제거
- 문자 아닌 특수문자 제거
- 단어 사이 공백 제거 : 띄어쓰기별로 split해주고 join

In [141]:
def data_cleansing(df):
    delete_email = re.sub(r'\b[\w\+]+@[\w]+.[\w]+.[\w]+.[\w]+\b', ' ', df)
    delete_number = re.sub(r'\b|\d+|\b', ' ',delete_email)
    delete_non_word = re.sub(r'\b[\W]+\b', ' ', delete_number)
    cleaning_result = ' '.join(delete_non_word.split())
    return cleaning_result 

In [142]:
news_df.loc[:, 'News'] = news_df['News'].apply(data_cleansing)
news_df.head()

,News,Target
0,From Mamatha Devineni Ratnam Subject Pens fans...,rec.sport.hockey
1,From edu Matthew B Lawson Subject Which high p...,comp.sys.ibm.pc.hardware
2,From hilmi Hilmi Eren Subject Re ARMENIA SAYS ...,talk.politics.mideast
3,From Guy Dawson Subject Re IDE vs SCSI DMA and...,comp.sys.ibm.pc.hardware
4,From Alexander Samuel McDiarmid Subject driver...,comp.sys.mac.hardware


**[ sklearn.feature_extraction.text ]**
- 데이터 전처리 과정의 tokenization, stopword제거, stemming등을 한번에
- Scikit-learn text vector화 모듈
- Text관련 처리를 Hyper parameter로 한번에 처리
- CountVectorizer, TfidVectorizer 제공
- Stemmer 등 NLTK등 외부모듈도 사용

**CountVectorizer**
- 문서 집합으로부터 단어의 수를 세어 카운트 행렬을 만듦<br><br>
![img](https://ifh.cc/g/5QPJd6.jpg)

**TfidfVectorizer**
- 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법
- TF(Term Frequency) : 문서에서 해당 단어가 얼마나 나왔는지 나타내주는 빈도 수
- DF(Document Frequency) : 해당 단어가 있는 문서의 수
- IDF(Inverse Document Frequency) 해당 단어가 있는 문서의 수가 높아질 수록 가중치를 축소해주기 위해 역수 취해줌
    - log(N/(1 + DF))
        - N : 전체 문서의 수
- TF-IDF : TF * IDF


- **CustomizedVectorizer** - StemmedCounterVectorizer, StemmedTfidfVectorizer

**[ Stemmer 적용하기 ]**
- Stemmer 또는 Lemma를 적용하기 위해 nltk사용
- 기존 Vectorizer의 상속을 받아 새로운 클래스 만들기

In [159]:
# nltk 설치 안되어있으면 커맨드창에서
# !conda install nltk
# 혹은
# !pip install nltk

Stemmer 사용예<br>
- look의 어간으로 stem처리하여 저장됨 

In [170]:
from nltk import stem
stmmer = stem.SnowballStemmer("english")
sentence = 'looking looks looked'
[stmmer.stem(word) for word in sentence.split()]

['look', 'look', 'look']

- 명사와 동사도 구분해줌

In [161]:
stmmer.stem("images"), stmmer.stem("imaging"), stmmer.stem("imagination")  

('imag', 'imag', 'imagin')

**StemmedCounterVectorizer**
- stemming기능을 추가한 Vectorizer 만들기

In [162]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
enlish_stemmer = nltk.stem.SnowballStemmer("english")
# CountVectorizer 상속받음
class StemmedCountVectorizer(CountVectorizer):
    # stemming 기능 추가
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        # 각각의 문서(doc)를 읽을 때, 단어를 하나씩 뽑아서 stem처리하고 반환해줘라
        return lambda doc: (enlish_stemmer.stem(w) for w in analyzer(doc))

In [171]:
# 두 Vectorizer의 결과가 약간씩 다름
print(StemmedCountVectorizer(min_df=1, stop_words="english").fit([sentence]).vocabulary_)
print(CountVectorizer(min_df=1, stop_words="english").fit([sentence]).vocabulary_)

{'look': 0}
{'looking': 1, 'looks': 2, 'looked': 0}


In [172]:
#TfidfVectorizer 구현
from sklearn.feature_extraction.text import TfidfVectorizer

enlish_stemmer = nltk.stem.SnowballStemmer("english")
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer,self).build_analyzer()
        return lambda doc: (enlish_stemmer.stem(w) for w in analyzer(doc))

**[ Scikit-learn Pipeline 모듈 ]**
- Train/ Test데이터에 적용할 과정들을 동일하게
- Vectorizer를 적용할 때는 Train/Test에 한번에
- 여러번 실험할 때 거쳐야 하는 과정을 정의
- Scikit-learn의 pipeline모듈 사용<br><br>
![img](https://ifh.cc/g/HNvDjS.png)<br><br>
![img](https://ifh.cc/g/GbShvQ.png)
- T1,T2라는 과정에 X,X1을 피팅시켜 저장하고 최종적으로 Classifer(Model)을 만들어냄
- 그 이후에 test데이터를 만나면 만들었던 Model로 predict를 진행

**Gridsearch**
- Hyperparameter를 search할 때 사용
- Hyperparameter 변수 조합 자동 생성하여 좋을 조합을 찾아내자
- Pipeline과 합쳐서 estimator별로 parameter설정
- GridsearchCV(CrossValidation)등 샘플링 class제공

**Pipeline + Gridsearch**<br>

![img](https://ifh.cc/g/FPyAlP.png)
- pipeline은 평소처럼만들고, 그 안의 vect,clf같은 parameter들을 param_grid에 올수 있는 인자들로 넣어서 최적의 조합을 찾아줌

<br><br><br>
**Modeling Plan**<br><br>
![img](https://ifh.cc/g/5N3HBG.png)
- 4개의 Vectorizer X 4개의 Algorithm => 16개의 Pipeline 생성
- CV는 시간관계상 2번만 진행

**Modeling**

In [174]:
from nltk import ngrams
sentence = 'this is a foo bar sentences and i want to ngramize it'
n = 6
sixgrams = ngrams(sentence.split(), 3)
for grams in sixgrams:
  print (grams)

('this', 'is', 'a')
('is', 'a', 'foo')
('a', 'foo', 'bar')
('foo', 'bar', 'sentences')
('bar', 'sentences', 'and')
('sentences', 'and', 'i')
('and', 'i', 'want')
('i', 'want', 'to')
('want', 'to', 'ngramize')
('to', 'ngramize', 'it')


- text들을 tokenize하다보면 데이터가 sparse하게 구성되어서 해당 데이터들을 csr이라는 형태로 저장된다.
- 그래서 이 유형의 데이터들을 **.toarray()** 등을 통해 numpy형태로 다르게 핸들링해야한다.

In [179]:
print(CountVectorizer().fit_transform([sentence]).toarray())
CountVectorizer().fit_transform([sentence])

[[1 1 1 1 1 1 1 1 1 1]]


<1x10 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [180]:
# GausianNB는 csr형태로 데이터를 못받고 dense한 형태의 데이터만 받을수 있어서 해당 내용에만 transform할 수 있도록 처리
from sklearn.base import TransformerMixin, BaseEstimator
class DenseTransformer(BaseEstimator, TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [181]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# vectorizer, algorithms종류들을 일단 리스트로 저장
vectorizer = [CountVectorizer(), TfidfVectorizer(), StemmedCountVectorizer(), StemmedTfidfVectorizer()]
# algorithms = [BernoulliNB(), MultinomialNB(), GaussianNB(), LogisticRegression()]
algorithms = [MultinomialNB(), LogisticRegression()]

pipelines  = [] 

# 경우의 수 만들어주기. 총 16개 생성
import itertools
for case in list(itertools.product(vectorizer, algorithms)):
    # GaussianNB형태만 다르게 학습하려고 if문
    if isinstance(case[1], GaussianNB):
        case = list(case)
        case.insert(1,  DenseTransformer())
    pipelines.append(make_pipeline(*case))
pipelines

[Pipeline(memory=None,
      steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(memory=None,
      steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
   ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False))]),
 Pipeline(memory=None,
      steps=[('tfi

- Vectorizer Common params 경우의 수

In [182]:
ngrams_params = [(1,1),(1,3)]
stopword_params = ["english"]
lowercase_params = [True, False]
# min(max)_df_params : 단어 등장빈도가 최소 몇%에서 최대 몇%까지만 피쳐생성
# 아래 경우에는 최소 10%에서 80%까지만 피쳐로 친다.
# linspace : 0.6에서 0.8까지를 4개의 구간으로 잘라서 4가지 linspace를 만들고 경우의 수를 만듦

max_df_params = np.linspace(0.6, 0.8, num=4) 
min_df_params = np.linspace(0.0, 0.1, num=1)

attributes = {"ngram_range":ngrams_params, "max_df":max_df_params,"min_df":min_df_params,
              "lowercase":lowercase_params,"stop_words":stopword_params}
vectorizer_names = ["countvectorizer","tfidfvectorizer","stemmedcountvectorizer","stemmedtfidfvectorizer"]
vectorizer_params_dict = {}

# 위의 attributes, vectorizer_names를 잘 조합해서 여러 종류의 vectorizer만듦
for vect_name in vectorizer_names:
    vectorizer_params_dict[vect_name] = {}
    for key, value in attributes.items():
        param_name = vect_name + "__" + key
        vectorizer_params_dict[vect_name][param_name] =  value

vectorizer_params_dict
#countervectorizer의 경우만 : ngram경우의수 : 3개
#                             max_df_params : 4개
#                             min_df_params : 4개
#                             lowercase : 2개(True,False)
#                             3 x 4 x 4 x 2 : 총 96개의 경우의 수


{'countvectorizer': {'countvectorizer__ngram_range': [(1, 1), (1, 3)],
  'countvectorizer__max_df': array([0.6       , 0.66666667, 0.73333333, 0.8       ]),
  'countvectorizer__min_df': array([0.]),
  'countvectorizer__lowercase': [True, False],
  'countvectorizer__stop_words': ['english']},
 'tfidfvectorizer': {'tfidfvectorizer__ngram_range': [(1, 1), (1, 3)],
  'tfidfvectorizer__max_df': array([0.6       , 0.66666667, 0.73333333, 0.8       ]),
  'tfidfvectorizer__min_df': array([0.]),
  'tfidfvectorizer__lowercase': [True, False],
  'tfidfvectorizer__stop_words': ['english']},
 'stemmedcountvectorizer': {'stemmedcountvectorizer__ngram_range': [(1, 1),
   (1, 3)],
  'stemmedcountvectorizer__max_df': array([0.6       , 0.66666667, 0.73333333, 0.8       ]),
  'stemmedcountvectorizer__min_df': array([0.]),
  'stemmedcountvectorizer__lowercase': [True, False],
  'stemmedcountvectorizer__stop_words': ['english']},
 'stemmedtfidfvectorizer': {'stemmedtfidfvectorizer__ngram_range': [(1, 1),


- Algorithms parameters 경우의 수

In [183]:
algorithm_names = ["bernoullinb","multinomialnb","gaussiannb","logisticregression"]
algorithm_names = ["multinomialnb", "logisticregression"]

algorithm_params_dict = {}


# Algorithms에 따라 조정해줄 parameter들이 조금씩 달라서 따로 지정해줌

#'bernoullinb', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))])
alpha_params = np.linspace(1.0, 1.0, num=1)
for i in range(1):
    algorithm_params_dict[algorithm_names[i]] = {
    algorithm_names[i]+ "__alpha" : alpha_params    
    }
# algorithm_params_dict[algorithm_names[2]] = {}


# LogisticRegression    
# multi_class : str, {‘ovr’, ‘multinomial’}, default: ‘ovr’
# C : float, default: 1.0
# solver : {‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’},
# n_jobs : int, default: 1
# penalty : str, ‘l1’ or ‘l2’, default: ‘l2’

# multi_class_params = ["ovr", "multinomial"]
c_params = [0.1,  5.0, 7.0, 10.0, 15.0, 20.0, 100.0]



algorithm_params_dict[algorithm_names[1]] = [{
    "logisticregression__multi_class" : ["multinomial"],
    "logisticregression__solver" : ["saga"],
    "logisticregression__penalty" : ["l1"],
    "logisticregression__C" : c_params
    },{
    "logisticregression__multi_class" : ["ovr"],
    "logisticregression__solver" : ['liblinear'],
    "logisticregression__penalty" : ["l2"],
    "logisticregression__C" : c_params
    }
    ]
algorithm_params_dict

{'multinomialnb': {'multinomialnb__alpha': array([1.])},
 'logisticregression': [{'logisticregression__multi_class': ['multinomial'],
   'logisticregression__solver': ['saga'],
   'logisticregression__penalty': ['l1'],
   'logisticregression__C': [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]},
  {'logisticregression__multi_class': ['ovr'],
   'logisticregression__solver': ['liblinear'],
   'logisticregression__penalty': ['l2'],
   'logisticregression__C': [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]}]}

- 이제 경우의 수 다만들었으니 pipeline에 각각의 알고리즘들을 넣어주자

In [184]:
pipeline_params= []
for case in list(itertools.product(vectorizer_names, algorithm_names)):
    vect_params = vectorizer_params_dict[case[0]].copy()
    algo_params = algorithm_params_dict[case[1]]
    
    if isinstance(algo_params, dict):
        # vect_paramsr에 algo_params를 합쳐(update)주고 append
        vect_params.update(algo_params)
        pipeline_params.append(vect_params)
    else:
        temp = []
        for param in algo_params:
            vect_params.update(param)
            temp.append(vect_params)
        pipeline_params.append(temp)
pipeline_params
# 출력값 해석

[{'countvectorizer__ngram_range': [(1, 1), (1, 3)],
  'countvectorizer__max_df': array([0.6       , 0.66666667, 0.73333333, 0.8       ]),
  'countvectorizer__min_df': array([0.]),
  'countvectorizer__lowercase': [True, False],
  'countvectorizer__stop_words': ['english'],
  'multinomialnb__alpha': array([1.])},
 [{'countvectorizer__ngram_range': [(1, 1), (1, 3)],
   'countvectorizer__max_df': array([0.6       , 0.66666667, 0.73333333, 0.8       ]),
   'countvectorizer__min_df': array([0.]),
   'countvectorizer__lowercase': [True, False],
   'countvectorizer__stop_words': ['english'],
   'logisticregression__multi_class': ['ovr'],
   'logisticregression__solver': ['liblinear'],
   'logisticregression__penalty': ['l2'],
   'logisticregression__C': [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]},
  {'countvectorizer__ngram_range': [(1, 1), (1, 3)],
   'countvectorizer__max_df': array([0.6       , 0.66666667, 0.73333333, 0.8       ]),
   'countvectorizer__min_df': array([0.]),
   'countvectorize

위 코드 결과 해석<br>
![img](https://ifh.cc/g/2msoNo.png)
- bernoullinb__alpha의 array안에 5개의 경우
- countervectorizer__lowercase : 2경우
- countervectorizer__max_df : 4경우
- countervectorizer__min_df : 4경우
- countervectorizer__ngram_range : 3경우
- 따라서 5경우 X 96경우 => 480번의 학습이 발생
- 거기다 CV=2로 설정했으니 총 960번의 학습이 발생한다.(pipeline_params의 1번째 경우만)


**Learn !**

In [188]:
from sklearn.preprocessing import LabelEncoder

# 시간 관계상 1000개의 데이터만 진행
X_data = news_df.loc[:, 'News'].tolist()[:1000]
y_data = news_df['Target'].tolist()[:1000]
y = LabelEncoder().fit_transform(y_data)

In [189]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# 물론 다른metric으로 scoring해도됨
scoring = ['accuracy']
estimator_results = []
for i, (estimator, params) in enumerate(zip(pipelines,pipeline_params)):
    n_jobs = 36
#     if i+1 % 3 == 0:
#         n_jobs = 2
    gs_estimator = GridSearchCV(
        # verbose : 중간중간 결과보여줄지여부
        # n_jobs : 몇개의 cpu코어를 사용할것인가
            refit="accuracy", estimator=estimator,param_grid=params, scoring=scoring, cv=5, verbose=1, n_jobs=n_jobs)
    print(gs_estimator)

    gs_estimator.fit(X_data, y)
    estimator_results.append(gs_estimator)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), p..., vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=36,
       param_grid={'countvectorizer__ngram_range': [(1, 1), (1, 3)], 'countvectorizer__max_df': array([0.6    , 0.66667, 0.73333, 0.8    ]), 'countvectorizer__min_df': array([0.]), 'countvectorizer__lowercase': [True, False], 'countvectorizer__stop_words': ['english'], 'multinomialnb__alpha': array([1.])},
       pre_dispatch='2*n_jobs', refit='accuracy',
       return_train_score='warn', scoring=['accuracy'], verbose=1)
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=36)]: Done  80 out of  80 | elapsed:   36.4s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=36,
       param_grid=[{'countvectorizer__ngram_range': [(1, 1), (1, 3)], 'countvectorizer__max_df': array([0.6    , 0.66667, 0.73333, 0.8    ]), 'countvectorizer__min_df': array([0.]), 'countvectorizer__lowercase': [True, False], 'countvectorizer__stop_words': ['english'], 'logisticregression__multi_class': [...ticregression__penalty': ['l2'], 'logisticregression__C': [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit='accuracy',
   

[Parallel(n_jobs=36)]: Done 128 tasks      | elapsed:  5.1min


In [ ]:
# 각각의 gridsearch의 결과물들을 넣어놓음
print(len(estimator_results))
estimator_results

- 결과들을 Dataframe으로 보기 위해 틀 만듦

In [ ]:
import pandas as pd
from pandas import DataFrame
result_df_dict = {}
result_attributes = ["vectorizer", "model", "accuracy", "recall_macro","precision_macro" , "min_df", 
                     "lowercase", "max_df", "binarize", "alpha", "ngram_range"
                     "multi_class", "penalty", "solver", "C"]

pieline_list =  list(itertools.product(vectorizer_names, algorithm_names))

for att in result_attributes:
    result_df_dict[att] = [None for i in range(16)]

result_df = DataFrame(result_df_dict)
result_df

- Dataframe안의 값을 estimator_results의 값을 활용하여 업데이트

In [ ]:
for i, estiamtor in enumerate(estimator_results):
    # estiamtor에서 가장 좋았던 estiamto와 그 index
    best_estimator = estiamtor.best_estimator_
    best_index = estiamtor.best_index_
    result_df_dict["vectorizer"][i] = pieline_list[i][0]
    result_df_dict["model"][i] = pieline_list[i][1]
    result_df_dict["accuracy"][i] = estiamtor.best_score_
#     result_df_dict["recall_micro"][i] = estiamtor.cv_results_["mean_test_recall_micro"][best_index]
#     result_df_dict["precision_micro"][i] = estiamtor.cv_results_["mean_test_precision_micro"][best_index]
    for key, value in estiamtor.best_params_.items():
        if key.split("__")[1] in result_df_dict:
            name = key.split("__")[1]
            result_df_dict[key.split("__")[1]][i] = value
#     print(estiamtor.best_params_)
#     print(a.named_steps)

In [ ]:
result_df = DataFrame(result_df_dict, columns=result_attributes)
result_df.sort_values("accuracy",ascending=False)